In [ ]:
# Install required libraries
%pip install -q transformers accelerate bitsandbytes torch

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

print("🔹 Loading Mistral model, this may take 3–5 minutes...")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

print("✅ Model loaded successfully with 4-bit quantization!")


🔹 Loading Mistral model, this may take 3–5 minutes...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

✅ Model loaded successfully with 4-bit quantization!


In [4]:
SAVE_DIR = "mistral_local"
print(f"💾 Saving model and tokenizer to {SAVE_DIR}...")

model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

print("✅ Model and tokenizer saved locally.")


💾 Saving model and tokenizer to mistral_local...
✅ Model and tokenizer saved locally.


In [5]:
# mistral_model.py

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Path to your locally saved model folder (downloaded from Colab)
MODEL_PATH = "./mistral_local"

print("🔹 Loading Mistral model from local directory (4-bit quantized)...")

# Configure 4-bit quantization for efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# Load model with quantization and device mapping
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto"
)

model.eval()
print("✅ Mistral model loaded successfully!")

# ---------------------- Response Generator ----------------------

def generate_response(prompt: str, max_new_tokens: int = 256):
    """
    Generate a natural language response from the Mistral model.
    """
    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

    except Exception as e:
        return f"⚠️ Error generating response: {str(e)}"


# ---------------------- Test ----------------------

if __name__ == "__main__":
    test_prompt = "Hello, how are you feeling today?"
    print("\n🧠 Testing model response...\n")
    response = generate_response(test_prompt)
    print("User:", test_prompt)
    print("Therapist Bot:", response)


🔹 Loading Mistral model from local directory (4-bit quantized)...


/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ Mistral model loaded successfully!

🧠 Testing model response...

User: Hello, how are you feeling today?
Therapist Bot: Hello, how are you feeling today? I’m feeling a little better, thank you for asking. I’ve been dealing with some health issues lately, but I’m trying to stay positive and focus on the things that make me happy.

One thing that always makes me happy is music. I love listening to all kinds of music, from classical to hip hop to indie rock. Music has a way of lifting my spirits and helping me get through tough times.

Another thing that makes me happy is spending time with my family and friends. I’m grateful for the people in my life who support me and make me laugh. I try to make time for them every day, even if it’s just a quick phone call or text message.

I also enjoy reading, hiking, and cooking. Reading takes me to new worlds and helps me learn new things. Hiking gets me outside and helps me clear my mind. Cooking allows me to be creative and experiment with new 